<a href="https://colab.research.google.com/github/weso500/IOT-Anomaly-Detection/blob/main/Single_Image_VAE_CB_V_GenLatent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Obtained intially from https://keras.io/examples/generative/vae/ and https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/generative/ipynb/vae.ipynb

In [1]:
pip install --upgrade keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.5 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1


In [2]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import tensorflow as tf
import keras
from keras import ops
from keras import layers

In [3]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.seed_generator = keras.random.SeedGenerator(1337)

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = ops.shape(z_mean)[0]
        dim = ops.shape(z_mean)[1]
        epsilon = keras.random.normal(shape=(batch, dim), seed=self.seed_generator)
        return z_mean + ops.exp(0.5 * z_log_var) * epsilon

Use this notebook as example for obtaining images: https://colab.research.google.com/drive/1EOFDisibXb9a4A3lZ_2z4Jbhlnx0OVy6 The below takes care of obtaining images

In [4]:
latent_dim = 3

encoder_inputs = keras.Input(shape=(256, 256, 3))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 128, 128, 32)   │            896 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 64, 64, 64)     │         18,496 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 262144)         │              0 │ conv2d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │     33,554,560 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ z_mean (Dense)            │ (None, 3)              │            387 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ z_log_var (Dense)         │ (None, 3)              │            387 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sampling (Sampling)       │ (None, 3)              │              0 │ z_mean[0][0],          │
│                           │                        │                │ z_log_var[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 33,574,726 (128.08 MB)

 Trainable params: 33,574,726 (128.08 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(64 * 64 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((64, 64, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(3, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 3)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 262144)              │       1,048,576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose (Conv2DTranspose)   │ (None, 128, 128, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_1 (Conv2DTranspose) │ (None, 256, 256, 32)        │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_2 (Conv2DTranspose) │ (None, 256, 256, 3)         │             867 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,104,835 (4.21 MB)

 Trainable params: 1,104,835 (4.21 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
from tensorflow.keras.layers import Layer
class KLLossLayer(Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))

        return kl_loss

class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.kl_loss_layer = KLLossLayer()

    def call(self, inputs):  # Add the call method
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        return reconstructed

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = ops.mean(
                ops.sum(
                    keras.losses.binary_crossentropy(data, reconstruction),
                    axis=(1, 2),
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - ops.square(z_mean) - ops.exp(z_log_var))
            kl_loss = ops.mean(ops.sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        kl_loss = self.kl_loss_layer([z_mean, z_log_var])
        return reconstructed, kl_loss, z_mean, z_log_var

    def predict(self, x):
        reconstructed, kl_loss, z_mean, z_log_var = super().predict(x)
        return reconstructed, kl_loss, z_mean, z_log_var


In [7]:
!unzip /content/Gold_Folder_Final.zip

Streaming output truncated to the last 5000 lines.
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/108Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/109Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/10Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/110Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/111Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/112Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/113Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/114Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/115Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGB

In [8]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Training_Data/Normal/Spectrograms_Normal_CB-V',
  image_size=(256, 256),
  batch_size=32,
  label_mode=None)


normalization_layer = layers.Rescaling(scale= 1./255)

normalized_ds = train_ds.map(lambda x: normalization_layer(x))

Found 500 files.


In [9]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-V',
  image_size=(256, 256),
  shuffle=False,
  batch_size=1,
  label_mode=None)

file_paths = test_ds.file_paths
print(file_paths)

normalization_layer = layers.Rescaling(scale= 1./255)

normalized_test_ds = test_ds.map(lambda x: normalization_layer(x))

Found 190 files.
['/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-V/500Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-V/501Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-V/502Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-V/503Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-V/504Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-V/505Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-V/506Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-V/507Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-V/508Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-V/509Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-V/510Normal.png', '/content/Gold_Folder_Final/Test_Dat

In [10]:
test_anomaly_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Test_Data/Anomaly/Spectrograms_Anomaly_CB-V',
  image_size=(256, 256),
  shuffle=False,
  batch_size=1,
  label_mode=None)

file_paths = test_ds.file_paths
print(file_paths)

normalization_layer = layers.Rescaling(scale= 1./255)

normalized_anomaly_test_ds = test_anomaly_ds.map(lambda x: normalization_layer(x))

Found 182 files.
['/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-V/500Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-V/501Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-V/502Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-V/503Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-V/504Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-V/505Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-V/506Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-V/507Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-V/508Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-V/509Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-V/510Normal.png', '/content/Gold_Folder_Final/Test_Dat

In [11]:
#(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
#mnist_digits = np.concatenate([x_train, x_test], axis=0)
#mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(normalized_ds, epochs=15, batch_size=32)

Epoch 1/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 27s 710ms/step - kl_loss: 603.4157 - loss: 43701.0938 - reconstruction_loss: 43097.6836
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 18s 85ms/step - kl_loss: 362.2431 - loss: 20093.1758 - reconstruction_loss: 19730.9316
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - kl_loss: 234.2659 - loss: 16901.7656 - reconstruction_loss: 16667.4980
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - kl_loss: 153.8917 - loss: 16266.5234 - reconstruction_loss: 16112.6318
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 92ms/step - kl_loss: 136.7859 - loss: 15350.7637 - reconstruction_loss: 15213.9785
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step - kl_loss: 225.0025 - loss: 13115.0049 - reconstruction_loss: 12890.0039
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 93ms/step - kl_loss: 197.1205 - loss: 11913.6152 - reconstruction_loss: 11716.4951
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 125ms/step - kl_loss: 157.9506 - loss: 11510.2090 - reconstruction_loss: 11352.2578
Epoc

In [12]:
train_ds_test_ = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Training_Data/Normal/Spectrograms_Normal_CB-V',
  image_size=(256, 256),
  batch_size=1,
  label_mode=None)


normalization_layer = layers.Rescaling(scale= 1./255)

normalized_ds_test1 = train_ds_test_.map(lambda x: normalization_layer(x))

Found 500 files.


In [13]:
pip install sewar

  Preparing metadata (setup.py) ... done
  Created wheel for sewar: filename=sewar-0.4.6-py3-none-any.whl size=11419 sha256=734db035b9639dceb7706f6f3ae565faab0d4a7c2b487f152ecde9a6ef476eb2
  Stored in directory: /root/.cache/pip/wheels/3f/af/02/9c6556ba287b62a945d737def09b8b8c35c9e1d82b9dfae84c
Successfully built sewar


In [14]:
import cv2
from urllib.request import AbstractBasicAuthHandler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp

#mse_values_normal = [] # Create an empty list to store mse values
#mse_values_anomaly = []
mse_values = []
kldiv1 = []
zmean1 = []
zmean2 = []
zmean3 = []
zlogvar1 = []
zlogvar2 = []
zlogvar3 = []

for elem in normalized_ds_test1.take(500):
  for i in range(len(elem)):
    elem_np = elem.numpy()
    exp = vae.predict(elem)
    exp_np = tf.get_static_value(exp)
    #plt.imshow(np.squeeze(elem_np))
    #plt.show()
    #plt.imshow(np.squeeze(exp_np[0]))
    #plt.show()
    #x = cv2.PSNR(elem_np, exp_np[0])
    x = mse(elem_np, exp_np[0])
    print(x)
    print(exp_np[1])
    kldiv1.append(exp_np[1])
    print(exp_np[2])
    zmean1.append(exp_np[2][0][0])
    zmean2.append(exp_np[2][0][1])
    zmean3.append(exp_np[2][0][2])
    print(exp_np[3])
    zlogvar1.append(exp_np[3][0][0])
    zlogvar2.append(exp_np[3][0][1])
    zlogvar3.append(exp_np[3][0][2])
    mse_values.append(x) # Append each mse value to the list



1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
0.0016328063486770671
56.97254
[[ 6.0327177 -5.0731473 -6.902344 ]]
[[-2.044387  -1.9678438 -2.8318706]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
0.0015997133407423796
58.447113
[[ 6.1158295 -5.1351686 -6.9927254]]
[[-2.0648527 -1.9810207 -2.8544621]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
0.0015912949893688525
58.210022
[[ 6.1094604 -5.121686  -6.977062 ]]
[[-2.053274  -1.9666904 -2.8366559]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
0.0016797102824929898
58.311394
[[ 6.1011333 -5.134559  -6.9846945]]
[[-2.0728102 -1.988472  -2.8685079]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
0.0016190613348928135
58.348915
[[ 6.1036377 -5.1355867 -6.9870443]]
[[-2.073231  -1.9895356 -2.8682928]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
0.0020215737753368156
56.754715
[[ 6.0001335 -5.070252  -6.8930683]]
[[-2.0695379 -2.0244243 -2.8774357]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
0.0017115986805550036
58.25788
[[ 6.0969152 -5.132598  -6.9817505]]
[[-2.0726943 -1.9925611 -2.870888

In [15]:
import pandas as pd

dict = {'MSE': mse_values, 'KL Loss': kldiv1, 'Zmean1': zmean1, 'Zmean2': zmean2, 'Zmean3': zmean3, 'Zlogvar1': zlogvar1,'Zlogvar2': zlogvar2, 'Zlogvar3': zlogvar3 }

df = pd.DataFrame(dict)

# saving the dataframe
df.to_csv('CB-V_train.csv')

In [16]:
import cv2
from urllib.request import AbstractBasicAuthHandler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp

#mse_values_normal = [] # Create an empty list to store mse values
#mse_values_anomaly = []
mse_values = []
kldiv1 = []
zmean1 = []
zmean2 = []
zmean3 = []
zlogvar1 = []
zlogvar2 = []
zlogvar3 = []

for elem in normalized_test_ds.take(190):
  for i in range(len(elem)):
    elem_np = elem.numpy()
    exp = vae.predict(elem)
    exp_np = tf.get_static_value(exp)
    #plt.imshow(np.squeeze(elem_np))
    #plt.show()
    #plt.imshow(np.squeeze(exp_np[0]))
    #plt.show()
    #x = cv2.PSNR(elem_np, exp_np[0])
    x = mse(elem_np, exp_np[0])
    print(x)
    print(exp_np[1])
    kldiv1.append(exp_np[1])
    print(exp_np[2])
    zmean1.append(exp_np[2][0][0])
    zmean2.append(exp_np[2][0][1])
    zmean3.append(exp_np[2][0][2])
    print(exp_np[3])
    zlogvar1.append(exp_np[3][0][0])
    zlogvar2.append(exp_np[3][0][1])
    zlogvar3.append(exp_np[3][0][2])
    mse_values.append(x) # Append each mse value to the list

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
0.0016909403828675384
56.766838
[[ 6.0314274 -5.05992   -6.887192 ]]
[[-2.031298  -1.9463123 -2.8074174]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
0.001675171560650066
56.89032
[[ 6.036939 -5.066036 -6.895154]]
[[-2.034981  -1.9485247 -2.8113365]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
0.0017874414505869261
56.79328
[[ 6.0266223 -5.0643263 -6.889736 ]]
[[-2.039107  -1.958848  -2.8219078]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
0.0017285553230525737
57.08777
[[ 6.0503807 -5.0723295 -6.9075923]]
[[-2.035446  -1.9472704 -2.8087947]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
0.0016079319879092173
58.01828
[[ 6.094406  -5.1159086 -6.9662595]]
[[-2.055498  -1.9698956 -2.8422933]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
0.0016198465381369199
58.39465
[[ 6.1145897 -5.1316843 -6.989542 ]]
[[-2.0618513 -1.9778445 -2.8500912]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.0016190524563065587
58.262512
[[ 6.1081004 -5.1250553 -6.9816437]]
[[-2.0597131 -1.9766116 -2.846173 ]]


In [17]:
import pandas as pd

dict = {'MSE': mse_values, 'KL Loss': kldiv1, 'Zmean1': zmean1, 'Zmean2': zmean2, 'Zmean3': zmean3, 'Zlogvar1': zlogvar1,'Zlogvar2': zlogvar2, 'Zlogvar3': zlogvar3 }

df2 = pd.DataFrame(dict)

# saving the dataframe
df2.to_csv('CB-V_testnorm.csv')

In [18]:
import cv2
from urllib.request import AbstractBasicAuthHandler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp

#mse_values_normal = [] # Create an empty list to store mse values
#mse_values_anomaly = []
mse_values = []
kldiv1 = []
zmean1 = []
zmean2 = []
zmean3 = []
zlogvar1 = []
zlogvar2 = []
zlogvar3 = []

for elem in normalized_anomaly_test_ds.take(190):
  for i in range(len(elem)):
    elem_np = elem.numpy()
    exp = vae.predict(elem)
    exp_np = tf.get_static_value(exp)
    #plt.imshow(np.squeeze(elem_np))
    #plt.show()
    #plt.imshow(np.squeeze(exp_np[0]))
    #plt.show()
    #x = cv2.PSNR(elem_np, exp_np[0])
    x = mse(elem_np, exp_np[0])
    print(x)
    print(exp_np[1])
    kldiv1.append(exp_np[1])
    print(exp_np[2])
    zmean1.append(exp_np[2][0][0])
    zmean2.append(exp_np[2][0][1])
    zmean3.append(exp_np[2][0][2])
    print(exp_np[3])
    zlogvar1.append(exp_np[3][0][0])
    zlogvar2.append(exp_np[3][0][1])
    zlogvar3.append(exp_np[3][0][2])
    mse_values.append(x) # Append each mse value to the list

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.0030217915802414146
54.97448
[[ 5.9480925 -4.973095  -6.773793 ]]
[[-1.982176  -1.8839813 -2.7322273]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
0.009977957098889838
53.5158
[[ 5.695562  -4.9347253 -6.7118654]]
[[-2.2141073 -2.5886304 -3.1620703]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.002746596263279577
54.96301
[[ 5.947332  -4.9727616 -6.7730265]]
[[-1.9821557 -1.8837385 -2.73226  ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
0.0016568595505582386
56.849518
[[ 6.0366077 -5.063613  -6.892151 ]]
[[-2.0320141 -1.9431226 -2.8066077]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
0.24788190993279355
93.14632
[[ 7.3933907 -5.9633646 -9.320972 ]]
[[-2.270356 -5.556468 -4.239793]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
0.002831001126984108
54.920067
[[ 5.9451404 -4.9706597 -6.7703824]]
[[-1.9810796 -1.8828522 -2.7307217]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
0.0016406469069130548
58.29139
[[ 6.1031437 -5.1309724 -6.983521 ]]
[[-2.0680504 -1.9872293 -2.862032 ]]
1/1 ━━

In [19]:
import pandas as pd

dict = {'MSE': mse_values, 'KL Loss': kldiv1, 'Zmean1': zmean1, 'Zmean2': zmean2, 'Zmean3': zmean3, 'Zlogvar1': zlogvar1,'Zlogvar2': zlogvar2, 'Zlogvar3': zlogvar3 }

df3 = pd.DataFrame(dict)

# saving the dataframe
df3.to_csv('CB-V_testanom.csv')

In [ ]:
from google.colab import files
files.download('/content/B+FLUX_testanom.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('/content/B+FLUX_testnorm.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('/content/B+_FLUX_train.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>